# Linked data en JSON-LD

## Inleiding

* MongoDB collection: verzameling *gelijksoortige documenten*
* Schema beschrijft dit "gelijksoortig":
    * structuur; veldnamen; types; waarden
* MongoDB query als "validator" gekoppeld aan collection
* alternatief: gebruik van JSON-schema
* gebruik van standaard-schema's - bijv. schema.org
    * vaste namen, met omschreven betekenis
    * -> documenten uitwisselbaar

## Hoe maak je dit expliciet?

* hoe maak je dit bruikbaar voor computers?
* hoe maak je duidelijk welke standaard-schema's je gebruikt?
* hoe geef je het verband aan met andere data?

## Linked data

Linked data volgens Tim Berners-Lee

1. Use URIs as names for things
2. Use HTTP URIs so that people can look up those names.
3. When someone looks up a URI, provide useful information, using the standards (RDF*, SPARQL)
4. Include links to other URIs. so that they can discover more things.

https://www.w3.org/DesignIssues/LinkedData.html

## (1) Gebruik URI's/IRI's als namen

URI/IRI: globale identificatie; onafhankelijk van lokale context

URI/IRI voorbeelden:

* urn:isbn:0-486-27557-4  (ISBN-nummer)
* urn:dev:mac:0024befffe804ff1 (MAC-adres)
* tel:+1-816-555-1212
* mailto:John.Doe@example.com
* ftp://ftp.is.co.za/rfc/rfc1808.txt
* http://www.ietf.org/rfc/rfc2396.txt

## (2) Gebruik HTTP URL's als URI's

Gebruik waar mogelijk HTTP URL's als URI's.

Toegepast op eerdere voorbeeld:

```
{
    "http://schema.org/name": "Harry van Doorn",
    "http://schema.org/email": "harryvdoorm@friendmail.org",
    "http://schema.org/telephone": "+31-6-1357 8642"
}
```

Omslachtig! -> compacter via JSON-LD

Gebruikte libraries:

* requests - versturen van HTTP-requests
* jsonld - json-ld bewerkingen (expand, compact)
* BeautifulSoup - selecteren van HTML-onderdelen

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from pyld import jsonld

## JSON-LD

JSON-LD: context definieert namen

```
{
  "@context: "http://schema.org",
  "name": "Harry van Doorn",
  "email": "harryvdoorm@friendmail.org",
  "telephone": "+31-6-1357 8642"
}
```

In [ ]:
mycontact = {
  "@context": "http://schema.org/docs/jsonldcontext.jsonld",
  "name": "Harry van Doorn",
  "email": "harryvdoorm@friendmail.org",
  "telephone": "+31-6-1357 8642"
}

expanded_contact = jsonld.expand(mycontact)
expanded_contact

jsonld.compact: compactificeren met context

In [ ]:
jsonld.compact(expanded_contact, "http://schema.org/docs/jsonldcontext.jsonld")

## (3) Geef extra informatie via de URL's

Welke gegevens bij "[http://schema.org/email](http://schema.org/email)"?

In [ ]:
r = requests.get("http://schema.org/email")
r.status_code

In [ ]:
r.text[0:500]

Via BeautifulSoup: jsonld-script selecteren

In [ ]:
soup =  BeautifulSoup(r.text, "html.parser")
jsonld_script = soup.find(type="application/ld+json")
jsonld_script

Selecteren van json-document uit script:

In [ ]:
json.loads(jsonld_script.contents[0].string)

### Functie get_json_ld

Functie voor ophalen json-ld uit html bij URL:

In [ ]:
def get_json_ld (url):
    r = requests.get(url)
    if r.status_code != 200: 
        return None
    soup =  BeautifulSoup(r.text, "html.parser")
    json_script = soup.find(type="application/ld+json")
    if json_script == None:
        return None
    return json.loads(json_script.contents[0].string)

## (4) Gebruik links in de gegevens bij een URL

* web: links voor verbinden van documenten in web
* data-web: links voor verbinden van data in data-web

## JSON-LD voor zoekmachines

JSON-LD in HTML voor zoekmachines

Google:  [Google structured data richtlijnen](https://developers.google.com/search/docs/guides/intro-structured-data)

Voorbeelden:

In [ ]:
get_json_ld("https://ieni.org")

In [ ]:
get_json_ld("https://www.smulweb.nl/recepten/941502/Gekookt-ei")

In [ ]:
get_json_ld("https://nl.wikipedia.org/wiki/Hedy_Lamarr")

## Het web van linked data

 [Linked Open Data Cloud](https://lod-cloud.net): linked open data 

* (Wikipedia) 
* [Wikidata](https://wikidata.org)
* [DBpedia](https://dbpedia.org)

Benaderen van deze gegevens via SPARQL etc.


**Linked data: eerste stap naar het Semantische Web**


### Nu zelf aan de slag...

## JSON-LD

In een JSON-object kun je namen gebruiken zoals "voornaam" en "adres".
Voor de menselijke lezer is vaak wel (ongeveer) duidelijk wat deze betekenen.
Maar bij ingewikkelder schema's is dat vaak minder duidelijk.

Door het gebruik van een schema voor een collection kun je ervoor zorgen dat de documenten in die collection in elk geval dezelfde namen gebruiken voor dezelfde begrippen. Dus bijvoorbeeld allemaal "givenName" in plaats van "firstName".

Als je de standaard-schema's van schema.org gebruikt als uitgangspunt weet je dat je aansluit op naamgeving die elders gebruikt wordt.

Met behulp van JSON-LD kun je dit nog een stap verder brengen: je koppelt de namen die je gebruikt direct aan de namen in (bijvoorbeeld) schema.org.

De namen van schema.org kun je toevoegen als *context* aan je JSON-objecten: de veldnaam `givenName` komt dan overeen met [http://schema.org/givenName](http://schema.org/givenName). Met andere woorden: een naam komt overeen met een (unieke) URL. Dit is één van de principes van het web: je gebruikt URIs (URLs) voor het *identificeren* van dingen ("resources") op een eenduidige manier. Omdat de naamgeving in het web wereldwijd dezelfde is, is voor iedereen duidelijk wat hier bedoeld wordt.

Via een context kun je een ook een lokale naam invoeren voor een begrip dat via een URL gedefinieerd is; bijvoorbeeld:

```
"schema": "http://schema.org/",
"voornaam": {"@id": "schema:givenName"}
```

Dit is een afkorting voor:

```
"voornaam": {"@id": "http://schema.org/givenName"}
```

Op deze manier kun je je eigen namen gebruiken, in de lokale taal, zonder de unieke identificatie van schema.org te verliezen.

> Dit maakt het gemakkelijker om aan te sluiten bij lokale gewoontes.

De namen van schema.org vormen niet alleen een unieke identificatie: via de URL kun je ook informatie over een naam ophalen. In de eerste plaats als gewoon HTML document, geschikt voor de menselijke lezer, maar daarnaast ook in een vorm die beter door machines verwerkt kan worden, in `json-ld` formaat. Deze beschrijving is te vinden in een apart script-gedeelte in het HTML-bestand.

